In [25]:
import pandas as pd

In [18]:
df = pd.read_csv('labelled_csv/annotated_corrected_with_contexts with llm labels - annotated_corrected_with_contexts.csv')

In [19]:
df.head()

,submissionId,submissionURL,submissionTitle,context1,context2,utterance,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?,llama ts_1,...,llama ts_4,llama ts_5,llama_mf,gemma2_ts_1,gemma2_ts_2,gemma2_ts_3,gemma2_ts_4,gemma2_ts_5,gemma2_mf,overall_mf
0,1dncran,https://i.redd.it/1xvccvq6oi8d1.jpeg,the clouds did a thing this morning...,looks like someone laid out some rails of blow,all the deities got together are they are havi...,colombian clouds,1,0,1,0,...,0,1,1,0,1,0,0,0,0,0
1,c9tdn7,https://i.imgur.com/6KzNep5.jpg,looks like i won't be listening to my new viny...,it’s even crammed in there so hard your mailbo...,is there's a way to ask for a refund from the ...,how can an assigned delivery person not know t...,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,kctb6e,https://i.redd.it/9knprglwx3561.jpg,sounded way better in my head,i hope we get our 9-month cupcake in a cup.,bruh wall-e's my fav pixar movie! take my upvo...,i remember when it was just supposed to be a m...,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
3,oagyyo,https://i.redd.it/dm6sx4nqn9871.jpg,was just trying to help the driver.,orders $10 in food. tries to leave $5.50 tip. ...,more that?,maybe it prevents accidentally doing the wrong...,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,faw3bs,https://i.imgur.com/gwVDXvB.jpg,wow,"holy shit, i've seen a lot of these but that i...",the really boring dorian grey,step 1: dress up as subject.\n\nstep 2: walk u...,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [20]:
columns_to_keep = ['submissionId','submissionTitle', 'utterance', 'isTextSarcastic?', 'llama_mf', 'gemma2_mf', 'overall_mf']
df = df[columns_to_keep]
df.head()

,submissionId,submissionTitle,utterance,isTextSarcastic?,llama_mf,gemma2_mf,overall_mf
0,1dncran,the clouds did a thing this morning...,colombian clouds,1,1,0,0
1,c9tdn7,looks like i won't be listening to my new viny...,how can an assigned delivery person not know t...,0,0,0,0
2,kctb6e,sounded way better in my head,i remember when it was just supposed to be a m...,0,0,0,0
3,oagyyo,was just trying to help the driver.,maybe it prevents accidentally doing the wrong...,0,0,0,0
4,faw3bs,wow,step 1: dress up as subject.\n\nstep 2: walk u...,1,1,1,1


# Evaluation

In [32]:
from sklearn.metrics import cohen_kappa_score
def compute_accuracy(y_true, y_pred):
    """
    Computes accuracy by comparing the true labels with the predicted labels.
    """
    correct_predictions = sum(y_t == y_p for y_t, y_p in zip(y_true, y_pred))
    accuracy = correct_predictions / len(y_true)
    return accuracy

def compute_precision(y_true, y_pred):
    """
    Computes precision by calculating the ratio of true positives to the sum of true and false positives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    predicted_positives = sum(y_p == 1 for y_p in y_pred)
    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    return precision

def compute_recall(y_true, y_pred):
    """
    Computes recall by calculating the ratio of true positives to the sum of true positives and false negatives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    actual_positives = sum(y_t == 1 for y_t in y_true)
    recall = true_positives / actual_positives if actual_positives > 0 else 0
    return recall

def compute_f1(precision, recall):
    """
    Computes F1 score as the harmonic mean of precision and recall.
    """
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def evaluate(y_true, y_pred):
    accuracy = compute_accuracy(y_true, y_pred)
    precision = compute_precision(y_true, y_pred)
    recall = compute_recall(y_true, y_pred)
    f1 = compute_f1(precision, recall)
    kappa = cohen_kappa_score(y_true, y_pred)

    print("\tAccuracy: ", accuracy, "\n", "\tPrecision: ", precision,"\n", "\tRecall: ", recall, "\n", "\tF1 score: ", f1,"\n", "\tCohen's Kappa Score: ", kappa)
    

## llama 3.1 evaluation results

In [29]:
evaluate(df['isTextSarcastic?'], df['llama_mf'])

	Accuracy:  0.7608695652173914 
 	Precision:  0.807843137254902 
 	Recall:  0.7715355805243446 
 	F1 score:  0.789272030651341 
 	Cohen's Kappa Score:  0.5132275132275133


## gemma 2 evaluation results

In [30]:
evaluate(df['isTextSarcastic?'], df['gemma2_mf'])

	Accuracy:  0.7489130434782608 
 	Precision:  0.7285067873303167 
 	Recall:  0.9044943820224719 
 	F1 score:  0.8070175438596491 
 	Cohen's Kappa Score:  0.4594512102066356


## Overall evaluation results

In [31]:
evaluate(df['isTextSarcastic?'], df['overall_mf'])

	Accuracy:  0.7684782608695652 
 	Precision:  0.7644151565074135 
 	Recall:  0.8689138576779026 
 	F1 score:  0.8133216476774758 
 	Cohen's Kappa Score:  0.5118621775390839
